<h2>Marketing Python Calculations</h2>

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("C:/Users/renan/Downloads/data_set_da_test/data_set_da_test.csv")

#Transforming event_date column to datetime formart
df['event_date'] = pd.to_datetime(df['event_date'])

print("Loading concluded!")


Loading concluded!


<h3>Arrange data to construct funnel</h3>

In [3]:
#Number of users who visited the website by counting the unique records in dataframe. -- WEBSITE VIEWS
visitors = df['user'].nunique()

#Number of users who navigated the website, by filtering per page types and counting unique user records. -- NAVIGATION
navigation = df.loc[(df.page_type == 'search_listing_page') | (df.page_type == 'listing_page')]['user'].nunique()

#Number of users who visualized product pages, by filtering per product_page and page_view and counting unique user records. -- PRODUCT VIEWS
product_views = df.loc[(df.page_type == 'product_page') & (df.event_type == 'page_view')]['user'].nunique() #only visualized product pages but didnt take an action

#Number of users who demonstrated intention to buy products, by filtering event_type as add_to_cart and counting unique user records. -- INTENTION TO BUY
intention_to_buy = df.loc[(df.event_type == 'add_to_cart')]['user'].nunique() #added a product to cart

#Number of users who purchased a product  by counting how many times the event appeared in the dataset. -- CHECKOUT
checkout = df.loc[(df.event_type == 'order')].value_counts().sum() #purchased a product

print(f" The number of visitors is: {visitors}")
print(f" The number of users who navigated the website : {navigation}")
print(f" The number of users who visualized product pages  : {product_views}")
print(f" The number of users who demonstrated intention to buy products  : {intention_to_buy}")
print(f" The number of users who purchased a product  : {checkout}")

 The number of visitors is: 288088
 The number of users who navigated the website : 168396
 The number of users who visualized product pages  : 142512
 The number of users who demonstrated intention to buy products  : 10060
 The number of users who purchased a product  : 8741


<h3>Plotting the funnel chart</h3>

In [5]:
from plotly import graph_objects as go

fig = go.Figure(go.Funnel(
    y = ["Visitors","Navigation", "Product Views", "Intention to Buy", "Checkout"],
    x = [visitors, navigation, product_views, intention_to_buy, checkout],
    textposition = "inside",
    textinfo = "value+percent initial",
    opacity = 0.65,
  
    )
    )

fig.update_layout(title = "Purchase Funnel AUTODOC", title_x = 0.5, autosize = False, width = 800, height = 800)
fig.show()

#fig.write_html("C:/Users/renan/Documents/Análises/Teste Empresa/funnel.html")

<h3>Conversion Overview</h3>

In [6]:
#Conversion Rate
conv_rate = round(checkout / visitors * 100, 1)

#Number of conversions per month
spm = df.loc[df["event_type"] == 'order']

september = spm.loc[spm['event_date'].dt.month == 9]
september = september.event_type.value_counts()

october = spm.loc[spm['event_date'].dt.month == 10]
october = october.event_type.value_counts()


print(f"The number of Conversions in September corresponds to {september} and in October is {october}")
print(f"The conversion rate is {conv_rate}%")



The number of Conversions in September corresponds to order    493
Name: event_type, dtype: int64 and in October is order    8248
Name: event_type, dtype: int64
The conversion rate is 3.0%


<h3>Average session time</h3>

In [10]:
#AVERAGE TIME PEOPLE SPEND ON THE WEBSITE
import datetime
time = df[['event_date','user','session', 'event_type']]
time['event_date'] = pd.to_timedelta(time['event_date'].dt.time.astype(str))
time = time['event_date'].mean()

print(f"The average time people navigate in the website is  : {time}")


The average time people navigate in the website is  : 0 days 14:31:53.319561921


C:\Users\renan\AppData\Local\Temp\ipykernel_30524\499605318.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<h3>Return Rate</h3>

In [11]:
#Return Rate
return_ = df[['user', 'session']]
return_ = return_.groupby(['user']).value_counts().reset_index()
return_ = return_.loc[return_[0] > 1]
return_ = return_['user'].value_counts().sum()
return_

return_cust_rate = round(return_ / visitors * 100, 1)
return_cust_rate

31.3

<h3>Products Overview</h3>

In [12]:
#MOST ADDED TO CART PRODUCTS

df_intention = df[(df['product'] != 0)]

top5 = df_intention.loc[(df_intention.event_type == 'add_to_cart')]['product'].value_counts().head()
bottom5 = df_intention.loc[(df_intention.event_type == 'add_to_cart')]['product'].value_counts().tail()

top5, bottom5

(26372760    21
 27133309    20
 27075197    18
 26780006    15
 20283956    13
 Name: product, dtype: int64,
 35518602    1
 28442152    1
 32753948    1
 19757387    1
 35704689    1
 Name: product, dtype: int64)

<h3>Return Rate</h3>

In [13]:
#Return Rate
return_ = df[['user', 'session']]
return_ = return_.groupby(['user']).count().reset_index()
return_ = return_.loc[return_['session'] > 1]
return_ = return_['user'].value_counts().sum()
return_

return_cust_rate = round(return_ / visitors * 100, 1)
return_cust_rate

31.6

<h3>Shopping Cart Abandonment Rate</h3>

In [14]:
#Shopping Cart Abandonment Rate

aban_rate = round((df.loc[(df.event_type == "order")].value_counts().sum() / df.loc[(df.event_type == "add_to_cart")].value_counts().sum()) * 100, 1)


aban_rate

54.6